## From WatchHistory.txt to FactWatches

In [ ]:
for index, row in df.iterrows():
    working_df = []
    
    # ...
    
    elif i[1] == "FactWatches":
        # SK_CUSTOMERID and SK_SECURITYID: slighly different (added "IsCurrent=1").
        # Put in as a comment as this is how it was done in FactWatches's historical load
        SK_CUSTOMERID = 0 #spark.sql("select SK_CustomerID from DimCustomer where CUSTOMERID = '" + row[0] + "'AND IsCurrent = 1")
        SK_SECURITYID = 0 #spark.sql("select SK_SecurityID from DimSecurity where SYMBOL = '" + row[1] + "'AND IsCurrent = 1")
        SK_DATEID_DATEPLACED=row[2]
        if row[3]=="ACTV":
            SK_DATEID_DATEREMOVED = ''
        else:
            SK_DATEID_DATEREMOVED = row[2]
        BATCHID=1

        working_df = [SK_CUSTOMERID,SK_SECURITYID,SK_DATEID_DATEPLACED,SK_DATEID_DATEREMOVED,BATCHID]

## From Prospect file to Prospect table

How did I perform the update?
1. Insert all the values in another table (e.g. "temp_Prospect")
2. When finished, check if there are values of "Prospect" not in "temp_Prospect". In case, insert them into "temp_Prospect". If you think about it, the output is exactly the same of the update.
3. Truncate table "Prospect"
4. Create table "Prospect" and copy the values from "temp_Prospect"
5. Truncate table "temp_Prospect"

In [ ]:
# Just after calling your .sql file to create the table, run this line to
# create the temporary table
spark.sql("CREATE TABLE temp_Prospect AS SELECT * FROM Prospect")

#...

for index, row in df.iterrows():
    working_df = []
    
    #...
    
    elif i[1] == "Prospect":
             
        isCustomer = spark.sql("select * from DimCustomer where status = 'ACTIVE' ")#to-do
        if isCustomer.count() == 0:
            isCustomer = 0
        else:
            isCustomer = 1
        SK_RecordDateID = 0 #to-do (both historical and incremental)
        SK_UpdateDateID = 0 #to-do (both historical and incremental)
        
        MarketingNameplate = ""
        if row[12] > 200000 or row[21] > 1000000:
            MarketingNameplate += "+HighValue"
        if row[14] > 3 or row[20] > 5:
            MarketingNameplate += "+Expenses"
        if row[16] > 45:
            MarketingNameplate += "+Boomer"
        if row[12] < 100000 or row[21] < 50000 or row[17] < 600:
            MarketingNameplate += "+MoneyAlert"
        if row[13] > 3 or row[20] > 7 :
            MarketingNameplate += "+Spender"
        if row[16] < 25 and row[12] > 1000000 :
            MarketingNameplate += "+Inherited"
        if len(MarketingNameplate) != 0:
            MarketingNameplate = MarketingNameplate[1:]
        
        
        working_df = [row[0],SK_RecordDateID,SK_UpdateDateID,1,isCustomer
                    ,row[1],row[2],row[3],row[4],row[5],row[6]
                    ,row[7],row[8],row[9],row[10]
                    ,row[11],row[12],row[13],row[14],row[15],row[16],
                    row[17],row[18],row[19],row[20],row[21],MarketingNameplate]
        print(MarketingNameplate)
        
        # ...

    csv_writer.writerow(working_df)

#add result 
sch = spark.sql("select * from " + i[1]).schema
df = spark.read.schema(sch).csv(os.getcwd() + "/result/temp_" + i[1] + ".csv", sep=',')
if i[1] != "Prospect"
    #df.write.mode("append").insertInto(i[1])
else:
    # Insert batch2's Prospect table values into the temporary table
    df.write.mode("append").insertInto("temp_Prospect")
    # Extract batch1's values not present in batch2
    input_query = """
    SELECT *
    FROM Prospect as p
    WHERE p.AgencyID NOT IN
    (
        SELECT tp.AgencyID
        FROM temp_Prospect as tp
    );
    """
    # Append them to the temporary table, which now contains "the updated"
    # values and the non-modified ones from Batch1
    spark.sql(input_query).write.insertInto("temp_Prospect", overwrite=False)
    # Erase all Prospect rows and copy temp_Prospect into it
    spark.sql("TRUNCATE TABLE Prospect")
    spark.sql("INSERT INTO Prospect SELECT * FROM temp_Prospect")
    # Erase all temp_Prospect rows
    spark.sql("TRUNCATE TABLE temp_Prospect")
    
    #...
    
    # At the start of the audit phase, erase "temp_Prospect"
    spark.sql("DROP TABLE IF EXISTS temp_Prospect")
    